# 🤖 Multilayer Perceptron

Esta Jupyter Notebook implementa una MLP para detectar patrones en una matriz 10x10.
Se genera un dataset para luego entrenar el modelo y validarlo.
Se evalúa la precisión para reiterar sobre el modelo con el objetivo de mejorar su eficacia.
Finalmente se exporta el modelo para ser utilizado en la aplicación web.


## 📋 Situación

En una matriz 10x10 se debe poder detectar las letras `b`, `d`, `f` que se corresponden a los siguientes patrones:

![Patrones que el MLP deberá reconocer](assets/patterns.png)

Ante un dato nuevo, el MLP deberá ser capaz de clasificar el contenido de esa matriz en uno de los tres patrones.


## 🗃️ Generación de Datasets

Conviene generar el conjunto de datos de manera programática.
Los datasets deberán ser representativos a la hora de definir la distribución de los ejemplos de entrenamiento.
Se definen los patrones 'b', 'd', y 'f'.
Luego, se crea una función `generate_sample` que crea un ejemplo de un patrón dado con una distorsión entre 0 y 0.3. 


In [10]:
from typing import Literal
import numpy as np

# Define the 10x10 patterns for 'b', 'd', and 'f'
PATTERNS = {
    "b": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
    "d": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
    "f": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
}


def generate_sample(
    pattern: Literal["b", "d", "f"], noise: float = 0.0
) -> np.ndarray[np.ndarray[int]]:
    """
    Generates a 10x10 matrix based on a given pattern letter ('b', 'd', or 'f'), with optional noise.
    Args:
        pattern: One of 'b', 'd', or 'f'.
        noise: Proportion of pixels to flip.
    Returns:
        A 10x10 numpy array with the pattern and noise applied.
    """
    matrix = PATTERNS[pattern].copy()
    num_pixels = matrix.size
    num_noisy = int(noise * num_pixels)

    if num_noisy > 0:
        # Choose random indices to flip
        indices = np.unravel_index(
            np.random.choice(num_pixels, num_noisy, replace=False), matrix.shape
        )
        # Flip the selected pixels (0 becomes 1, 1 becomes 0)
        matrix[indices] = 1 - matrix[indices]

    return matrix


print("A 'b' pattern with zero noise:")
print(generate_sample("d", 0.00))
print()
print("A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:")
print(generate_sample("d", 0.30))

A 'b' pattern with zero noise:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:
[[1 0 0 0 1 0 0 0 0 1]
 [1 0 1 0 0 1 0 1 1 0]
 [0 0 1 0 0 0 1 1 0 0]
 [1 1 0 1 0 0 0 1 0 1]
 [1 0 1 1 0 0 1 1 0 0]
 [0 0 1 1 0 1 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 1]
 [1 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 1 1 1 1 1 0]
 [1 0 0 0 0 0 1 0 0 0]]


Luego se generan 3 datasets que contengan 100, 500 y 1000 ejemplos. 
El 10% serán patrones sin distorsionar y el resto con una distorsión del 1% al 30%.
Se usará una **distribución uniforme** para ubicar el 90% de ejemplos en el rango de distorsión entre 0.01 y 0.30.

Cada dataset será un `pd.DataFrame` de `pandas` que contiene columnas del `0` al `99` (una por cada celda de la matriz) y una columna final `class` que indica la clase del patrón ('b', 'd' o 'f').


In [11]:
import pandas as pd

# Extend maximum width when printing DataFrames so they fit in just one line
pd.options.display.width = 100


def generate_dataset(n_samples: int) -> pd.DataFrame:
    """
    Generates a dataset of pattern samples.
    The 90% of samples will have noise between 0.01 and 0.30.

    Args:
        n_samples: Number of samples to generate.
    Returns:
        A dataframe with 100 columns for the flattened pattern and 1 column 'class' for the pattern class.
    """
    columns = [str(i) for i in range(100)] + ["class"]
    df = pd.DataFrame(0, index=np.arange(n_samples), columns=columns)
    df = df.astype({"class": "str"})

    for i in range(n_samples):
        # 10% without distortion, 90% with distortion between 1% and 30%
        if i < int(0.1 * n_samples):
            noise = 0.0
        else:
            noise = np.random.uniform(0.01, 0.30)

        # Pick a pattern at random
        pattern = np.random.choice(list(PATTERNS.keys()))

        sample = generate_sample(pattern, noise).flatten()
        df.iloc[i, :100] = sample
        df.loc[i, "class"] = pattern

    return df


df_100 = generate_dataset(100)
df_500 = generate_dataset(500)
df_1000 = generate_dataset(1000)

print("El dataset de 1000 ejemplos es el siguiente:")
print(df_1000)

El dataset de 1000 ejemplos es el siguiente:
     0  1  2  3  4  5  6  7  8  9  ...  91  92  93  94  95  96  97  98  99  class
0    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
1    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
2    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
3    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
4    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
..  .. .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..    ...
995  0  0  0  0  0  0  1  0  1  1  ...   0   1   0   0   0   1   0   0   1      f
996  0  0  0  1  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   1      f
997  0  0  1  0  1  1  0  0  0  0  ...   0   1   0   0   0   0   0   0   0      b
998  0  0  0  1  0  0  0  0  1  1  ...   0   0   0   0   0   0   0   0   1      d
999  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   

Por cada dataset deberán construirse tres conjuntos de validación con 10%, 20% y 30% de los ejemplos.
Esto da un total de 9 pares de datasets de entrenamiento y datasets de validación:

1. Dataset original de 100 ejemplos con 90% para entrenamiento y 10% para validación.
2. Dataset original de 100 ejemplos con 80% para entrenamiento y 20% para validación.
3. Dataset original de 100 ejemplos con 70% para entrenamiento y 30% para validación.
4. Dataset original de 500 ejemplos con 90% para entrenamiento y 10% para validación.
5. Dataset original de 500 ejemplos con 80% para entrenamiento y 20% para validación.
6. Dataset original de 500 ejemplos con 70% para entrenamiento y 30% para validación.
7. Dataset original de 1000 ejemplos con 90% para entrenamiento y 10% para validación.
8. Dataset original de 1000 ejemplos con 80% para entrenamiento y 20% para validación.
9. Dataset original de 1000 ejemplos con 70% para entrenamiento y 30% para validación.

In [13]:
VALIDATION_RATIOS = [0.1, 0.2, 0.3]


def split_dataset(df: pd.DataFrame, validation_ratio: float, random_state: int = None):
    """
    Splits the dataset into training and validation sets.

    Args:
        df: The dataset to split.
        validation_ratio: Proportion (between 0 and 1) of the dataset to include in the validation set.
        random_state (optional): Seed for reproducibility.
    Returns:
        tuple: (training_df, validation_df)
    """
    np.random.seed(random_state)
    shuffled_df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    validation_size = int(len(df) * validation_ratio)
    validation_df = shuffled_df.iloc[:validation_size].reset_index(drop=True)
    training_df = shuffled_df.iloc[validation_size:].reset_index(drop=True)
    return training_df, validation_df


# Store all datasets in one dictionary
datasets = {}
for df_name, df in [("100", df_100), ("500", df_500), ("1000", df_1000)]:
    datasets[df_name] = {}
    for validation_ratio in VALIDATION_RATIOS:
        training, validation = split_dataset(df, validation_ratio, random_state=42)
        key = f"val_{int(validation_ratio * 100)}%"
        # Store training dataset and validation dataset for each dataset, for each validation split
        datasets[df_name][key] = {
            "training": training,
            "validation": validation,
        }

print(datasets)

{'100': {'val_10%': {'training':     0  1  2  3  4  5  6  7  8  9  ...  91  92  93  94  95  96  97  98  99  class
0   0  0  0  0  0  0  1  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
1   0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
2   0  0  0  0  0  0  1  0  0  1  ...   0   0   1   0   0   0   1   0   0      f
3   0  0  1  0  0  1  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
4   0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
.. .. .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..    ...
85  0  0  0  0  0  0  1  1  0  0  ...   0   0   0   1   0   1   0   1   1      d
86  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   1   0   0      b
87  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
88  0  0  0  0  0  0  0  0  0  0  ...   1   0   1   0   0   1   0   0   0      b
89  0  0  0  0  1  0  0  1  0  0  ...   1   0   0   0   0   0   0   0   0   